In [2]:
# ! pip install tensorflow  tensorflow-gpu

In [3]:
# ! pip list

In [4]:
import tensorflow as tf
import os

In [5]:
# to avoid OOM errors by setting GPU Memory consumption Growth
# gpus=tf.config.experimental.list_physical_devices("CPU")

In [6]:
# len(gpus)

REMOVE DODGY IMAGE

In [7]:
import cv2
import imghdr

In [8]:
data_dir="img"

In [9]:
image_ext=["jpeg","jpg","png","bmp"]

In [10]:
os.listdir(data_dir)

['happy people', 'sad people']

In [11]:
# os.listdir(os.path.join(data_dir,"happy people"))

In [12]:
for classes in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir,classes)):
        img_path=os.path.join(data_dir,classes,image)
        try:
            im=cv2.imread(img_path)
            tip=imghdr.what(img_path)
            if tip not in image_ext:
                print(tip)
                print(f"image not in ext list{img_path}")
                os.remove(img_path)
                
        except Exception as e:
            print("issue")

In [13]:
tf.data.Dataset.list_files

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.list_files(file_pattern, shuffle=None, seed=None, name=None) -> 'DatasetV2'>

In [14]:
import numpy as np
import matplotlib.pyplot as plt

In [15]:
data=tf.keras.utils.image_dataset_from_directory("img")


Found 329 files belonging to 2 classes.


In [16]:
data_iter=data.as_numpy_iterator()

In [17]:
# to get another batch of data from iterator
batch=data_iter.next()

In [18]:
batch[0][:4]

array([[[[169.15234   , 143.07617   ,  82.10156   ],
         [175.68555   , 146.45703   ,  86.38086   ],
         [185.52344   , 152.76172   ,  91.634766  ],
         ...,
         [ 68.61914   ,  27.253906  ,   2.8730469 ],
         [ 69.        ,  27.        ,   3.        ],
         [ 68.02539   ,  26.02539   ,   2.0253906 ]],

        [[169.15234   , 143.07617   ,  82.10156   ],
         [175.68555   , 146.45703   ,  86.38086   ],
         [185.52344   , 152.76172   ,  91.634766  ],
         ...,
         [ 68.09906   ,  27.773983  ,   2.8730469 ],
         [ 68.4043    ,  26.495049  ,   2.4496727 ],
         [ 68.01027   ,  26.010265  ,   2.0102654 ]],

        [[169.14406   , 143.72852   ,  82.427734  ],
         [175.38422   , 147.10938   ,  86.70703   ],
         [185.48203   , 153.37265   ,  91.96094   ],
         ...,
         [ 67.74609   ,  28.126953  ,   2.5054665 ],
         [ 67.69867   ,  26.45367   ,   1.75      ],
         [ 67.991714  ,  26.008282  ,   1.6738281 ]],

In [19]:
fig,ax=plt.subplots(ncols=4,figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

PREPROSSING


In [20]:
#sclale data
s_data=data.map(lambda x,y:(x/255,y))

In [21]:
# converting batches of data to list


In [22]:
len(data)

11

In [23]:
train_size=int(len(s_data)*.7)
val_size=int(len(s_data)*.2)
test_size=int(len(s_data)*.1)

In [24]:
train=s_data.take(train_size)
val=s_data.skip(train_size).take(val_size)
test=s_data.skip(train_size).skip(val_size).take(test_size)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense,Flatten,Dropout

In [26]:
model=Sequential()

In [27]:
model.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1,activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

C:\Users\chand\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [28]:
model.compile('adam',loss=tf._losses.BinaryCrossentropy(),metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 16)   │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 60, 60, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 30, 30, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 14400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     3,686,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,692,001 (14.08 MB)

 Trainable params: 3,692,001 (14.08 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
logdir="logs"

In [31]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [32]:
hist=model.fit(train,epochs=35,validation_data=val,callbacks=[tensorboard_callback])

Epoch 1/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 487ms/step - accuracy: 0.5330 - loss: 0.7640 - val_accuracy: 0.5156 - val_loss: 0.7059
Epoch 2/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 468ms/step - accuracy: 0.5463 - loss: 0.6652 - val_accuracy: 0.5625 - val_loss: 0.6290
Epoch 3/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 449ms/step - accuracy: 0.6338 - loss: 0.6079 - val_accuracy: 0.6719 - val_loss: 0.5359
Epoch 4/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 536ms/step - accuracy: 0.6742 - loss: 0.6011 - val_accuracy: 0.6719 - val_loss: 0.5649
Epoch 5/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 479ms/step - accuracy: 0.7823 - loss: 0.5169 - val_accuracy: 0.7969 - val_loss: 0.4393
Epoch 6/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 472ms/step - accuracy: 0.8278 - loss: 0.4126 - val_accuracy: 0.7188 - val_loss: 0.4435
Epoch 7/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 484ms/step - accuracy: 0.8176 - loss: 0.3834 - val_accuracy: 0.9062 - val_loss: 0.3254
Epoch 8/35
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 492ms/step - accuracy: 0.8858 - loss: 0.3163 - val_accuracy: 0.8594 - val_loss:

In [33]:
hist.history

{'accuracy': [0.5401785969734192,
  0.5669642686843872,
  0.6383928656578064,
  0.6830357313156128,
  0.7857142686843872,
  0.8080357313156128,
  0.8080357313156128,
  0.8616071343421936,
  0.8928571343421936,
  0.875,
  0.9598214030265808,
  0.9642857313156128,
  0.9553571343421936,
  0.9598214030265808,
  0.9732142686843872,
  0.9821428656578064,
  0.9821428656578064,
  0.9866071343421936,
  0.9776785969734192,
  0.9866071343421936,
  0.9776785969734192,
  0.9821428656578064,
  0.9910714030265808,
  0.9910714030265808,
  0.9955357313156128,
  0.9866071343421936,
  0.9821428656578064,
  0.9910714030265808,
  0.9910714030265808,
  0.9866071343421936,
  0.9776785969734192,
  0.9910714030265808,
  0.9910714030265808,
  0.9821428656578064,
  0.9955357313156128],
 'loss': [0.7598479390144348,
  0.6626787185668945,
  0.6015914082527161,
  0.5854426622390747,
  0.516626238822937,
  0.42998477816581726,
  0.38227444887161255,
  0.31677135825157166,
  0.26117897033691406,
  0.28096023201942444

In [34]:
plt.plot(hist.history["loss"],color="red",label="loss")
plt.plot(hist.history['val_loss'],color="yellow",label="val_loss")

In [35]:
d=s_data.as_numpy_iterator().next()
all_items = [item for item in d]
items = [list(row) for row in zip(*all_items)]
cv2.imshow("im",items[0][0])
cv2.waitKey(0)
cv2.destroyAllWindows()
plt.imshow(items[1][0])

In [36]:
i=items[13][0]
model.predict(np.expand_dims(i/255,0))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


array([[0.99816865]], dtype=float32)

In [37]:
items[13][1]

0

In [38]:
plt.imshow(i)

In [39]:
len(items)

32

In [40]:
from tensorflow.keras.metrics import Precision,Recall,BinaryAccuracy

In [41]:
pre=Precision()
re=Recall()
acc=BinaryAccuracy()

In [42]:
for batch in test.as_numpy_iterator():
    X,y=batch
    yhat=model.predict(X)
    if yhat[0]<0.5:
        print("yes",yhat[0])
    else:
        print("no",yhat[0])
    print(y,yhat)
    pre.update_state(y,yhat)
    re.update_state(y,yhat)
    acc.update_state(y,yhat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
yes [0.00070362]
[0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 0 0] [[7.0361933e-04]
 [7.1341597e-04]
 [9.9999994e-01]
 [2.7719815e-03]
 [1.0000000e+00]
 [1.2246455e-03]
 [1.8422017e-03]
 [9.9918032e-01]
 [7.0286205e-04]
 [3.5813167e-03]
 [5.9719408e-05]
 [9.9945039e-01]
 [1.4147344e-02]
 [7.6372535e-03]
 [9.9911648e-01]
 [9.9994743e-01]
 [1.4689673e-02]
 [9.9467009e-01]
 [4.7719432e-04]
 [1.4150417e-02]
 [9.9988705e-01]
 [2.9002414e-03]
 [6.6762447e-01]
 [7.2853621e-03]
 [9.9758077e-01]
 [1.8831224e-05]
 [9.9998468e-01]
 [3.8543000e-04]
 [9.9980706e-01]
 [9.9899203e-01]
 [5.4879900e-04]
 [2.6670804e-03]]


In [43]:
print(pre.result().numpy(),re.result().numpy(),acc.result().numpy())

0.9230769 1.0 0.96875


In [44]:
model.predict(np.expand_dims(i/255,0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


array([[0.99816865]], dtype=float32)

In [45]:
im=cv2.imread(r"images (7).jpeg")
imr=tf.image.resize(im,(256,256))
print(im.shape)
plt.imshow(im)
print(imr.shape)
plt.imshow(imr.numpy().astype(int))

(201, 251, 3)
(256, 256, 3)


In [46]:
yh=model.predict(np.expand_dims(imr/255,0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


array([[0.9995766]], dtype=float32)